In [56]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as partial
%matplotlib inline

In [83]:
# 导入训练数据和测试数据
df_train = pd.read_csv("train_data_public.csv")
df_test = pd.read_csv("test_public.csv")
df_train = df_train.drop(['id'],axis=1)
df_test = df_test.drop(['id'],axis=1)
df_train.head()

,text,BIO_anno,class
0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1


In [74]:
# # 切分训练集和验证集
# ratio = 0.8
# offset = int(ratio*len(df_train))
# df_dev = df_train.iloc[offset:].copy()
# df_train = df_train.iloc[:offset].copy()
# df_dev.head()

In [93]:
# 利用jieba库完成中午分词
import jieba

jieba_text = []
for x in df_train['text']:
    jieba_text.append(jieba.lcut(x))

emo_data = np.c_[jieba_text,df_train['class']]

Building prefix dict from the default dictionary ...
DEBUG 2022-08-25 18:30:32,080 __init__.py:113] Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
DEBUG 2022-08-25 18:30:32,944 __init__.py:146] Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.020 seconds.
DEBUG 2022-08-25 18:30:33,103 __init__.py:164] Loading model cost 1.020 seconds.
Prefix dict has been built successfully.
DEBUG 2022-08-25 18:30:33,104 __init__.py:166] Prefix dict has been built successfully.


['工商银行',
 '的',
 '信用卡',
 '是',
 '经典',
 '信用卡',
 '中',
 '的',
 '常青树',
 '，',
 '在',
 '工行',
 '区',
 '的',
 '讨论',
 '热度',
 '也',
 '是',
 '常年',
 'TOP',
 ' ',
 '1',
 '。']